In [3]:
import py_entitymatching as em
import pandas as pd
import numpy as np
import os

In [4]:
datasets_dir = em.get_install_path() + os.sep + 'datasets'
path_A = datasets_dir + os.sep + 'tracks_sample.csv'
path_B = datasets_dir + os.sep + 'songs_sample.csv'
path_G = datasets_dir + os.sep + 'labeled_data.csv'

In [5]:
# Read the CSV files
A = em.read_csv_metadata(path_A,key='id', low_memory=False) # setting the parameter low_memory to False  to speed up loading.
B = em.read_csv_metadata(path_B,key='id',low_memory=False)
G = em.read_csv_metadata(path_G,key='id',low_memory=False,ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [6]:
# Split G into I an J
train_test = em.split_train_test(G, train_proportion=0.5,random_state=0)
I = train_test['train']
J = train_test['test']
I.to_csv('train.csv')
J.to_csv('test.csv')

In [310]:
# Generate a set of features
F = em.get_features_for_matching(A, B)
print(F.feature_name)
#Step 1 removing all features on id parameters
AB = F[4:]
#AB = F.iloc[[7,8,16]]
#print(type(F))
#Step 2 - removing year_year_exm, year_year_lev_dist and Step 3 removing year_year_anm
AB = AB.drop(AB.index[[0,1,2]])
# #Step 4 - removing song_title_song_title_lev_dist, song_title_song_title_nmw and song_title_song_title_sw
# AB = AB.drop(AB.index[[5,7,8]])
# #Step 5 - removing song_title_song_title_cos_dlm_dc0_dlm_dc0, song_title_song_title_mel
# AB = AB.drop(AB.index[[2,4]])
# #Step 6 - removing song_title_song_title_jac_dlm_dc0_dlm_dc0, song_title_song_title_lev_sim
# AB = AB.drop(AB.index[[2,3]])
# #Step 7 - removing artists_artists_lev_dist, artists_artists_nmw,artists_artists_sw
# AB = AB.drop(AB.index[[3,4,5,6,8,9]])
# #Step 8 - removing artists_artists_lev_sim
# AB = AB.drop(AB.index[[3]])


0                                     id_id_exm
1                                     id_id_anm
2                                id_id_lev_dist
3                                 id_id_lev_sim
4                                 year_year_exm
5                                 year_year_anm
6                            year_year_lev_dist
7                             year_year_lev_sim
8         song_title_song_title_jac_qgm_3_qgm_3
9     song_title_song_title_cos_dlm_dc0_dlm_dc0
10    song_title_song_title_jac_dlm_dc0_dlm_dc0
11                    song_title_song_title_mel
12               song_title_song_title_lev_dist
13                song_title_song_title_lev_sim
14                    song_title_song_title_nmw
15                     song_title_song_title_sw
16              artists_artists_jac_qgm_3_qgm_3
17          artists_artists_cos_dlm_dc0_dlm_dc0
18          artists_artists_jac_dlm_dc0_dlm_dc0
19                          artists_artists_mel
20                     artists_artists_l

In [311]:
AB.feature_name

7                             year_year_lev_sim
8         song_title_song_title_jac_qgm_3_qgm_3
9     song_title_song_title_cos_dlm_dc0_dlm_dc0
10    song_title_song_title_jac_dlm_dc0_dlm_dc0
11                    song_title_song_title_mel
12               song_title_song_title_lev_dist
13                song_title_song_title_lev_sim
14                    song_title_song_title_nmw
15                     song_title_song_title_sw
16              artists_artists_jac_qgm_3_qgm_3
17          artists_artists_cos_dlm_dc0_dlm_dc0
18          artists_artists_jac_dlm_dc0_dlm_dc0
19                          artists_artists_mel
20                     artists_artists_lev_dist
21                      artists_artists_lev_sim
22                          artists_artists_nmw
23                           artists_artists_sw
Name: feature_name, dtype: object

In [312]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=AB, 
                            attrs_after='gold_labels',
                            show_progress=False) 

In [313]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.notnull(H))

True

In [314]:
# Impute feature vectors with the mean of the column values.
# H = em.impute_table(H, 
#                 exclude_attrs=['id', 'ltable_id', 'rtable_id', 'gold_labels'],
#                 strategy='mean')

H.fillna(value=0, inplace=True)

In [315]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0,max_depth=5)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In [316]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['id', 'ltable_id', 'rtable_id', 'gold_labels'],
        k=5,
        target_attr='gold_labels', metric='precision', random_state=0)
# result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
#         exclude_attrs=['id', 'ltable_id', 'rtable_id', 'gold_labels'],
#         k=5,
#         target_attr='gold_labels', metric='recall', random_state=0)
# result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
#         exclude_attrs=['id', 'ltable_id', 'rtable_id', 'gold_labels'],
#         k=5,
#         target_attr='gold_labels', metric='f1', random_state=0)
result['cv_stats']
# result['cv_stats'].to_csv("results.csv")

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x00000250C4581160>,5,0.833333,0.958333,1.000000,0.956522,0.882353,0.926108
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x00000250C4581C18>,5,0.888889,0.920000,1.000000,0.958333,1.000000,0.953444
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x00000250C4581400>,5,0.607143,0.821429,0.870968,0.833333,0.678571,0.762289
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x00000250C4581978>,5,0.850000,0.920000,0.964286,0.962963,0.950000,0.929450
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x00000250C4581BE0>,5,0.850000,0.916667,0.931034,1.000000,1.000000,0.939540
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x00000250C4581908>,5,0.888889,0.920000,0.962963,0.923077,1.000000,0.938986


In [317]:
#Debug Random Forest
# Split H into P and Q
PQ = em.split_train_test(H, train_proportion=0.5, random_state=0)
P = PQ['train']
Q = PQ['test']

In [ ]:
# Debug RF matcher using GUI
em.vis_debug_rf(rf, P, Q, 
        exclude_attrs=['id', 'ltable_id', 'rtable_id', 'gold_labels'],
        target_attr='gold_labels')

In [300]:
#Step 4 - removing song_title_song_title_lev_dist, song_title_song_title_nmw and song_title_song_title_sw
# AB = AB.drop(AB.index[[5,7,8]])
#Step 5 - removing song_title_song_title_cos_dlm_dc0_dlm_dc0, song_title_song_title_mel
AB = AB.drop(AB.index[[2,4]])
#Step 6 - removing song_title_song_title_jac_dlm_dc0_dlm_dc0, song_title_song_title_lev_sim
AB = AB.drop(AB.index[[2,3]])
#Step 7 - removing artists_artists_lev_dist, artists_artists_nmw,artists_artists_sw
AB = AB.drop(AB.index[[3,4,5,6,8,9]])
#Step 8 - removing artists_artists_lev_sim
AB = AB.drop(AB.index[[3]])

In [301]:
H = em.extract_feature_vecs(I, 
                            feature_table=AB, 
                            attrs_after='gold_labels',
                            show_progress=False) 

In [302]:
# Step 9 Convert the I into a set of feature vectors using F
H['song_title_song_title_jac_qgm_3_qgm_3']
H['artists_artists_jac_qgm_3_qgm_3']
H['song_title_artists_score']= H.song_title_song_title_jac_qgm_3_qgm_3*H.artists_artists_jac_qgm_3_qgm_3

In [303]:
# Impute feature vectors with the mean of the column values.
# H = em.impute_table(H, 
#                 exclude_attrs=['id', 'ltable_id', 'rtable_id', 'gold_labels'],
#                 strategy='mean')

H.fillna(value=0, inplace=True)

In [304]:
#Debug Random Forest
# Split H into P and Q
PQ = em.split_train_test(H, train_proportion=0.5, random_state=0)
P = PQ['train']
Q = PQ['test']

In [305]:
# Debug RF matcher using GUI
em.vis_debug_rf(rf, P, Q, 
        exclude_attrs=['id', 'ltable_id', 'rtable_id', 'gold_labels'],
        target_attr='gold_labels')

In [211]:
# Evaluating matching output
#Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=AB,
                            attrs_after='gold_labels', show_progress=False)

In [212]:
# Check if the feature vectors contain missing values
# A return value of True means that there are missing values
any(pd.notnull(L))
L.fillna(value=0, inplace=True)

In [64]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['id', 'ltable_id', 'rtable_id', 'gold_labels'], 
       target_attr='gold_labels')

In [213]:
# Train using feature vectors from I 
lg.fit(table=H, 
       exclude_attrs=['id', 'ltable_id', 'rtable_id', 'gold_labels'], 
       target_attr='gold_labels')
# Predict on L 
predictions = lg.predict(table=L, exclude_attrs=['id', 'ltable_id', 'rtable_id', 'gold_labels'], 
                         append=True,target_attr='predicted_labels')
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_labels', 'predicted_labels')
em.print_eval_summary(eval_result)

ValueError: X has 3 features per sample; expecting 4

In [66]:
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['id', 'ltable_id', 'rtable_id', 'gold_labels'], 
                         append=True,target_attr='predicted_labels')

ValueError: Number of features of the model must match the input. Model n_features is 17 and input n_features is 18 

In [38]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_labels', 'predicted_labels')
em.print_eval_summary(eval_result)

Precision : 94.55% (104/110)
Recall : 90.43% (104/115)
F1 : 92.44%
False positives : 6 (out of 110 positive predictions)
False negatives : 11 (out of 90 negative predictions)
